# 0. Table of content:

[1. Case study, second part assignment:](#1.-Case-study,-second-part-assignment:)  
[2.Database connection:](#2.Database-connection:)  
[3. Questions:](#3._Questions:)  
    - [Question 10:](#Question-10:)  
    - [Question 11:](#Question-11:)  
    - [Question 12:](#Question-12:)  
    - [Question 13:](#Question-13:)  


# 1. Case study, second part assignment:

**PART 2: SQLite**  

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook for the following questions.   
  
**QUESTIONS:**  

**Q10: Produce a list of facilities with a total revenue less than 1000.**  
The output of facility name and total revenue, sorted by revenue. Remember  
that there's a different cost for guests and members!  
  
**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order.**  
  
**Q12: Find the facilities with their usage by member, but not guests.**  
  
**Q13: Find the facilities usage by month, but not guests**  

# 2.Database connection:

In [7]:
from sqlalchemy import create_engine
import requests
import pandas as pd

In [8]:
# url = "https://github.com/BenoitLoze1977/19_3_sql_casestudy/raw/master/data/raw/sqlite_db_pythonsqlite.db"
# output_path = "sqlite_db_pythonsqlite.db"

# # Download the file
# r = requests.get(url)
# with open(output_path, "wb") as f:
#     f.write(r.content)

# print("Database downloaded successfully.")

Database downloaded successfully.


In [9]:
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

# 3. Questions:

## Question 10:

In [12]:
query10 = """
SELECT
	sub.facility_name,
    SUM(total_cost) AS total_revenue
FROM (
	SELECT
		f.name AS facility_name,
		CASE
			WHEN b.memid = 0
			THEN f.guestcost * b.slots
			ELSE f.membercost * b.slots
		END AS total_cost
	FROM Bookings AS b
	INNER JOIN Facilities AS f
	ON b.facid = f.facid
	INNER JOIN Members AS m
	ON b.memid = m.memid
    ) AS sub
GROUP BY sub.facility_name
HAVING total_revenue < 1000
ORDER BY total_revenue;
"""

# Run the query and load the result into a DataFrame
df = pd.read_sql_query(query10, engine)

# Display the result
df.head()

,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


## Question 11:

In [14]:
# SQLite supports string concatenation using the || operator instead of CONCAT_WS().
query11 = """
SELECT
    m1.surname || ', ' || m1.firstname AS member,
    m2.surname || ', ' || m2.firstname AS recommended_by
FROM Members AS m1
JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER by member;
"""

# Run the query and load the result into a DataFrame
df = pd.read_sql_query(query11, engine)

# Display the result
df.head()

,member,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"


## Question 12:

In [16]:
query12 = """
SELECT
	f.name,
    COUNT(*) AS member_usage_count
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
INNER JOIN Members AS m
ON b.memid = m.memid
WHERE b.memid <> 0
GROUP BY f.name
ORDER BY member_usage_count DESC;
"""

# Run the query and load the result into a DataFrame
df = pd.read_sql_query(query12, engine)

# Display the result
df.head()


,name,member_usage_count
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344


## Question 13:

In [18]:
# SQLite does not support DATE_FORMAT():
query13 = """
SELECT
	f.name AS facility_name,
    STRFTIME('%Y-%m', b.starttime) AS month,
    COUNT(*) AS member_usage_count
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
INNER JOIN Members AS m
ON b.memid = m.memid
WHERE b.memid <> 0
GROUP BY f.name, STRFTIME('%Y-%m', b.starttime)
ORDER BY month DESC, member_usage_count DESC;
"""

# Run the query and load the result into a DataFrame
df = pd.read_sql_query(query13, engine)

# Display the result
df.head()

,facility_name,month,member_usage_count
0,Pool Table,2012-09,408
1,Snooker Table,2012-09,199
2,Table Tennis,2012-09,194
3,Massage Room 1,2012-09,191
4,Badminton Court,2012-09,161
